In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import itertools
import cv2
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report

In [56]:
pwd

'/workspace/aman/MANYA'

In [57]:
df = pd.read_csv('GroundTruth.csv')
df.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
df.shape

(10015, 8)

In [59]:
dfBKL = df[df["BKL"]==1]

In [60]:
dfBKL.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
6,ISIC_0024312,0.0,0.0,0.0,0.0,1.0,0.0,0.0
18,ISIC_0024324,0.0,0.0,0.0,0.0,1.0,0.0,0.0
30,ISIC_0024336,0.0,0.0,0.0,0.0,1.0,0.0,0.0
31,ISIC_0024337,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32,ISIC_0024338,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [61]:
dfBKL.drop(columns=["MEL", "NV", "AKIEC", "BCC", "DF", "VASC"], inplace=True)

<ipython-input-61-d2cbce2e4940>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfBKL.drop(columns=["MEL", "NV", "AKIEC", "BCC", "DF", "VASC"], inplace=True)


In [62]:
dfBKL.head()

,image,BKL
6,ISIC_0024312,1.0
18,ISIC_0024324,1.0
30,ISIC_0024336,1.0
31,ISIC_0024337,1.0
32,ISIC_0024338,1.0


In [63]:
dfNotBKL = df[df["BKL"]==0]

In [64]:
dfNotBKL.tail()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
10008,ISIC_0034314,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10010,ISIC_0034316,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10011,ISIC_0034317,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10013,ISIC_0034319,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10014,ISIC_0034320,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [65]:
dfNotBKL.drop(columns=["MEL", "NV", "AKIEC", "BCC", "DF", "VASC"], inplace=True)

<ipython-input-65-28e79ebf3a5a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNotBKL.drop(columns=["MEL", "NV", "AKIEC", "BCC", "DF", "VASC"], inplace=True)


In [66]:
dfNotBKL.head()

,image,BKL
0,ISIC_0024306,0.0
1,ISIC_0024307,0.0
2,ISIC_0024308,0.0
3,ISIC_0024309,0.0
4,ISIC_0024310,0.0


In [67]:
dfNotBKL.shape

(8916, 2)

In [68]:
dfBKL.shape

(1099, 2)

In [69]:
dfNotBKL = dfNotBKL.sample(frac = 0.2, random_state = 40)

In [70]:
dfBKL.shape

(1099, 2)

In [71]:
dfNotBKL.shape

(1783, 2)

In [72]:
df = pd.concat([dfBKL, dfNotBKL])

In [73]:
df.head()

,image,BKL
6,ISIC_0024312,1.0
18,ISIC_0024324,1.0
30,ISIC_0024336,1.0
31,ISIC_0024337,1.0
32,ISIC_0024338,1.0


In [74]:
df.shape

(2882, 2)

In [75]:
df = df.sample(frac=1, random_state=30)

In [76]:
df.head()

,image,BKL
8662,ISIC_0032968,0.0
2735,ISIC_0027041,1.0
9203,ISIC_0033509,1.0
1102,ISIC_0025408,0.0
8845,ISIC_0033151,0.0


In [77]:
imageArray = df["image"].to_numpy()

In [78]:
imageArray.shape

(2882,)

In [79]:
pwd

'/workspace/aman/MANYA'

In [80]:
cd images

/workspace/aman/MANYA/images


In [87]:
pwd

'/workspace/aman/MANYA/images'

In [88]:
Images = []

In [89]:
def processImage(image):
    image = cv2.resize(image, (150, 200))
    image = image.astype("float32") / 255
    return image

In [90]:
for img in imageArray:
    path = img + ".jpg"
    image = cv2.imread(path)
    image = processImage(image)
    Images.append(image)

In [91]:
len(Images)

2882

In [92]:
pwd

'/workspace/aman/MANYA/images'

In [93]:
np.save("BKLIMAGES.npy", Images)

In [94]:
output = df["BKL"].to_numpy()

In [95]:
len(output)

2882

In [96]:
output.shape

(2882,)

In [97]:
xTrain, xTest, yTrain, yTest = train_test_split(np.array(Images), output, test_size = 0.2, random_state = 30)

In [98]:
xTrain.shape

(2305, 200, 150, 3)

In [99]:
yTrain.shape

(2305,)

In [100]:
xTrain, xVal, yTrain, yVal = train_test_split(xTrain, yTrain, test_size = 0.2, random_state = 30)

In [101]:
xVal.shape

(461, 200, 150, 3)

In [102]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=(200, 150, 3)))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 150, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 150, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 75, 32)       0         
_________________________________________________________________
dropout (Dropout)            (None, 100, 75, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 75, 64)       18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 75, 64)       36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 37, 64)        0

In [103]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["Accuracy"])

In [104]:
history = model.fit(xTrain, yTrain, validation_data = (xVal, yVal), epochs = 10, batch_size = 32)

Epoch 1/10
58/58 [==============================] - 7s 71ms/step - loss: 1.9577 - accuracy: 0.0000e+00 - val_loss: 0.5987 - val_accuracy: 0.0000e+00
Epoch 2/10
58/58 [==============================] - 2s 37ms/step - loss: 0.5842 - accuracy: 0.0000e+00 - val_loss: 0.5383 - val_accuracy: 0.0000e+00
Epoch 3/10
58/58 [==============================] - 2s 34ms/step - loss: 0.5159 - accuracy: 0.0000e+00 - val_loss: 0.5627 - val_accuracy: 0.0000e+00
Epoch 4/10
58/58 [==============================] - 2s 33ms/step - loss: 0.5352 - accuracy: 0.0000e+00 - val_loss: 0.5397 - val_accuracy: 0.0000e+00
Epoch 5/10
58/58 [==============================] - 2s 30ms/step - loss: 0.5202 - accuracy: 0.0000e+00 - val_loss: 0.5194 - val_accuracy: 0.0000e+00
Epoch 6/10
58/58 [==============================] - 2s 30ms/step - loss: 0.4986 - accuracy: 0.0000e+00 - val_loss: 0.5158 - val_accuracy: 0.0000e+00
Epoch 7/10
58/58 [==============================] - 2s 30ms/step - loss: 0.4900 - accuracy: 0.0000e+00 - v

In [105]:
predictions = model.predict(xTest)

In [106]:
preds = []

In [107]:
for p in predictions:
    if p>=0.5:
        preds.append(1)
    else:
        preds.append(0)

In [108]:
report = classification_report(yTest, preds, target_names = ["NOT NV", "NV"])

In [109]:
print(report)

              precision    recall  f1-score   support

      NOT NV       0.82      0.76      0.79       349
          NV       0.67      0.75      0.71       228

    accuracy                           0.76       577
   macro avg       0.75      0.76      0.75       577
weighted avg       0.76      0.76      0.76       577

